<mark>**Please ensure all capacities in your Fabric tenant are running before running this script**</mark>

**Please enter the name of the lakehouse where the final Metadata table will be stored in the cell below**

In [18]:
%%configure -f
{
    "defaultLakehouse": {  
        "name": "lakehouse03"
    }
}

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, -1, Finished, Available, Finished)

**Please specify variables for lakehouse, warehouse below**

In [19]:
#Specify variables
myLakehouse = 'lakehouse03'
myWorkspace = 'WS_SagarFabric03'
myTablePath = "abfss://"+myWorkspace+"@onelake.dfs.fabric.microsoft.com/"+myLakehouse+".Lakehouse/Tables/ShortcutsMetadata"
#print(myTablePath)

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 3, Finished, Available, Finished)

In [20]:
pip install jsonmerge

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 4, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [21]:
import json, requests, pandas as pd, jsonmerge
import datetime
from io import StringIO
import numpy as np


StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 5, Finished, Available, Finished)

In [22]:
# Generate token for Fabric access
access_token = mssparkutils.credentials.getToken('pbi')

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 6, Finished, Available, Finished)

In [23]:
# Function definitions to query Fabric REST API

# List workspaces in current tenant
def listWorkspaces(access_token):
     base_url = 'https://api.fabric.microsoft.com/v1/admin/workspaces?state=Active&type=Workspace'
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)
     data=response.json()
     jsondata = json.loads(json.dumps(response.text))
     #df = spark.read.json(sc.parallelize([jsondata]))
     return jsondata

# List items of specific type inside of a workspace
def listWorkspaceItems(access_token,workspaceId,itemType):
     base_url = f"https://api.fabric.microsoft.com/v1/admin/items?workspaceId={workspaceId}&type={itemType}"
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)   
     data=response.json()
     jsondata = json.loads(json.dumps(response.text))
     #df = spark.read.json(sc.parallelize([data]))
     return jsondata
    
# Retrieve shortcut information for individual shortcuts
def getShortcutInfo(access_token,workspaceId,lakehouseId,shortcutName):
     base_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{lakehouseId}/shortcuts/Tables/{shortcutName}"
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)
     data=response.json()
     return data

# List all tables inside of a specific Lakehouse
def listLakeHouseTables(access_token,workspaceId,lakehouseId):
     base_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}/tables"
     header = {"Authorization": f"Bearer {access_token}"}
     response = requests.get(base_url, headers=header)
     data=response.json()
     return data

# List all file shortcuts inside of a specific Lakehouse
def listFileShortcuts(access_token,workspaceId,lakehouseId):
     base_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{lakehouseId}/shortcuts"
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)
     data=response.json()
     return data

# List all shortcuts inside of a specific Lakehouse
def listallShortcuts(access_token,workspaceId,lakehouseId):
     base_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{lakehouseId}/shortcuts"
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)
     data=response.json()
     jsondata = json.loads(json.dumps(response.text))
     #df = spark.read.json(sc.parallelize([data]))
     return jsondata

# Get item details based on an item id
def getitemdetails(access_token,workspaceId,itemId):
     base_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}"
     header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
     response = requests.get(base_url, headers=header)
     data=response.json()
     jsondata = json.loads(json.dumps(response.text))
     #df = spark.read.json(sc.parallelize([data]))
     return jsondata

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 7, Finished, Available, Finished)

**The cell below uses listWorkspaces, listWorkspaceItems and listallShortcuts functions defined above to get all shortcuts information. listWorkspaceItems gets all Lakehouses within a given workspace. listallShortcuts then iterates through all the lakehouses and gets all shortcuts within each lakehouse**

**The reason the code below uses mergejson and not dataframe joins is because all shortcut details do not have the lakehouse/workspace id to join the 3 dataframes (workspace, lakehouse and shortcuts). Hence we have loop through each workspace, lakehouse and shortcut and append relevant content together**

In [24]:
from io import StringIO

# define dataframe for all shortcuts details
df_sh_all = pd.DataFrame()

#get all workspaces and iterate through them
df_ws = pd.read_json(StringIO(listWorkspaces(access_token)))
for index, row in df_ws.iterrows():

    # Get the workspace details to add to the shortcuts info
    workspace_id = (row['workspaces']['id'])
    workspace_name = (row['workspaces']['name'])
    workspace_capacity_id = (row['workspaces']['capacityId'])

    # Get all lakehouses in a workspace and iterate through them 
    df_lh = pd.read_json(StringIO(listWorkspaceItems(access_token,workspace_id,"Lakehouse")))

    # If no lakehouses in a workspace, create a record for the df_sh_all with no lakehouse and shortcut data
    if df_lh.empty:
        newrow = {'value':None,
                  'workspace_id':workspace_id,
                  'workspace_name':workspace_name,
                  'workspace_capacity_id':workspace_capacity_id,
                  'lakehouse_id':None,
                  'lakehouse_name':None,
                  'target_shortcut_type':None,
                  'target_shortcut_name':None
                 }
        #display(newrow)
        df_sh_all = df_sh_all._append(newrow, ignore_index=True)
        #display(df_sh_all)

    else:# iterate through lakehouses

        for index, row in df_lh.iterrows():

            # Get the lakehouse details to add to the shortcuts info
            lakehouse_id = (row['itemEntities']['id'])
            lakehouse_name = (row['itemEntities']['name'])
        
            newrow = {'workspace_id':workspace_id,
                      'workspace_name':workspace_name,
                      'workspace_capacity_id':workspace_capacity_id,
                      'lakehouse_id':lakehouse_id,
                      'lakehouse_name':lakehouse_name
                      }
            #display(newrow)      
            #display("****************")

            # Get all shortcuts in a lakehouse and iterate through them 
            df_sh = pd.read_json(StringIO(listallShortcuts(access_token,workspace_id,lakehouse_id)))

            # If no shortcuts in the lakehouse, create a record for the df_sh_all with no shortcut data
            if df_sh.empty:
                
                newrow = {'value':None,
                          'workspace_id':workspace_id,
                          'workspace_name':workspace_name,
                          'workspace_capacity_id':workspace_capacity_id,
                          'lakehouse_id':lakehouse_id,
                          'lakehouse_name':lakehouse_name,
                          'target_shortcut_type':None,
                          'target_shortcut_name':None
                         }
                df_sh_all = df_sh_all._append(newrow, ignore_index=True)      

            else:# iterate through shortcuts
            
                #for index, row in df_sh.iterrows(): 
                for i in df_sh.index:
                    #display(row['value'])   

                    # get details of workspace/lakehouse captured above, convert to json dict
                    js_ws_lh = json.loads(json.dumps(newrow))
                    #display("Workspace and Lakehouse info where shortcut is created") 
                    #display(js_ws_lh)
                    #display("****************")                    

                    # get all details for a shortcut, convert to json dict
                    js_sh = df_sh.iloc[i].to_json()
                    js_sh = json.loads(js_sh)
                    #display("Shortcut details")
                    #display(js_sh)
                    #display("****************")
                    #js_ws_lh = json.loads(newrow)

                    # get type of the target Onelake item shortcut points to. It can be a lakehouse/warehouse/KQL database
                    # Do this only for Onelake
                    if js_sh['value']['target']['type'] == 'OneLake':
                        #get the itemid and workspace id of the shortcut target
                        target_item_id = js_sh['value']['target']['oneLake']['itemId']
                        target_workspace_id = js_sh['value']['target']['oneLake']['workspaceId']
                        #display("Onelake item id")
                        #display(target_item_id)
                        #get the target shortcuts details
                        df_itemdetails = pd.read_json(StringIO(getitemdetails(access_token,target_workspace_id,target_item_id)),lines=True)
                        #display(df_itemdetails)
                        #get only the item type and name
                        df_itemdetails = df_itemdetails[['type','displayName']]
                        #display(df_itemdetails)
                        df_itemdetails = df_itemdetails.rename(columns={'type':'target_shortcut_type','displayName': 'target_shortcut_name'})
                        #display(df_itemdetails)

                    else:
                        #if not onelake shortcut, create an empty dataframe for the target shortcut details
                        df_itemdetails = pd.DataFrame([[np.nan, np.nan]], columns=['target_shortcut_type','target_shortcut_name'])
                        #display(df_itemdetails)


                    js_items = df_itemdetails.iloc[0].to_json()
                    js_items = json.loads(js_items) 
                    #display("item details")
                    #display(js_items)
                    #display("****************")    

                    # merge the 2 json dicts
                    merg = jsonmerge.merge(js_sh, js_ws_lh)
                    #display('merg')
                    #display(merg)
                    merg1 = jsonmerge.merge(merg,js_items)          
                    #display('merg1')
                    #display(merg1)                                              
                    #join to the final dataframe having all the information
                    #df_sh_all = df_sh_all.join(df_itemdetails)

                    # merge the 2 json dicts
                    #merg = jsonmerge.merge(js_sh, js_ws_lh)
                    df_sh_all = df_sh_all._append(merg1, ignore_index=True)
                    #display(df_sh_all)

    #print("===========================================")   

display(df_sh_all)


StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d7e3fe26-7f28-4a79-8eac-1c388dddff8e)

In [25]:
# Convert pandas dataframe to spark dataframe and load into a view for further processing
df_sh_all_spark = spark.createDataFrame(df_sh_all)
df_sh_all_spark.createOrReplaceTempView("vw_allShortcuts_01")

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 9, Finished, Available, Finished)

In [26]:
%%sql
CREATE OR REPLACE TEMP VIEW vw_allShortcuts_02 AS
SELECT workspace_id, workspace_name, workspace_capacity_id, lakehouse_id, lakehouse_name, target_shortcut_type, target_shortcut_name, value.*
FROM vw_allShortcuts_01

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

%%sql
select * from vw_allShortcuts_02

In [27]:
%%sql
CREATE OR REPLACE TEMP VIEW vw_allShortcuts_03 AS
SELECT workspace_id, workspace_name, workspace_capacity_id, lakehouse_id, lakehouse_name,
       name AS shortcut_name, path AS shortcut_path, target.type AS shortcut_type,
       CASE WHEN target.type = 'OneLake'
            THEN target.oneLake.workspaceId
            ELSE NULL
       END AS target_workspace_id,
       CASE WHEN target.type = 'OneLake'
            THEN target_shortcut_type
            ELSE target.type
       END AS target_oneLake_shortcut_type,
       CASE WHEN target.type = 'OneLake'
            THEN target_shortcut_name
            ELSE NULL
       END AS target_oneLake_shortcut_location,
       target.oneLake.path AS target_oneLake_shortcut_name,
       to_json(target) AS shortcut_details
FROM vw_allShortcuts_02

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

%%sql
CREATE OR REPLACE TEMP VIEW vw_allShortcuts_03 AS
SELECT workspace_id, workspace_name, workspace_capacity_id, lakehouse_id, lakehouse_name,
       name AS shortcut_name, path AS shortcut_path, target.type AS shortcut_type,
       CASE WHEN target.type = 'OneLake'
            THEN target.oneLake.workspaceId
            ELSE NULL
       END AS target_workspace_id,
       target_shortcut_type AS target_oneLake_shortcut_type,
       CASE WHEN target.type = 'OneLake'
            THEN target_shortcut_name
            WHEN target.type = 'adlsGen2' AND instr(get_json_object(to_json(target), '$.adlsGen2'),'') > 0 = true
            THEN concat(target.adlsGen2.location,target.adlsGen2.subpath)
            WHEN target.type = 'amazonS3' AND instr(get_json_object(to_json(target), '$.amazonS3'),'') > 0 = true
            THEN concat(target.amazonS3.location,target.amazonS3.subpath)
            --WHEN target.type = 'S3Compatible' AND EXISTS(instr(get_json_object(to_json(target), '$.s3Compatible'),'') > 0)
            --THEN target.s3Compatible.location+target.s3Compatible.subpath        
            --WHEN target.type = 'GoogleCloudStorage' AND EXISTS(instr(get_json_object(to_json(target), '$.googleCloudStorage'),'') > 0)
            --THEN target.googleCloudStorage.location+target.googleCloudStorage.subpath                 
            ELSE NULL
       END AS target_shortcut_location,
       target_shortcut_name AS target_oneLake_shortcut_name,
       to_json(target) AS shortcut_details
FROM vw_allShortcuts_02


%%sql
select* from vw_allShortcuts_03

In [28]:
# Convert the workspaces pandas dataframe to spark dataframe and load into a view. This is for getting the name of the 'target' workspace which has the table/file which the shortcut points to
df_ws_spark = spark.createDataFrame(df_ws)
#display(df_sh_all_spark)
df_ws_spark.createOrReplaceTempView("vw_workspaces")

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 12, Finished, Available, Finished)

In [29]:
df_final = spark.sql("""SELECT a.*, b.workspaces.name AS target_workspace_name, CURRENT_TIMESTAMP AS LoadDatetime
                      FROM vw_allShortcuts_03 a
                      LEFT JOIN vw_workspaces b
                      ON a.target_workspace_id = b.workspaces.id""")


StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 13, Finished, Available, Finished)

In [30]:
#Drop and recreate the final table
if spark.catalog.tableExists("ShortcutsMetadata"):
    drop_stmt = 'DROP TABLE '+myLakehouse+'.ShortcutsMetadata'
    result = spark.sql(drop_stmt)
df_final.write.format("delta").mode("overwrite").save(myTablePath)

StatementMeta(, 4a50c71f-7d67-42f3-a1ef-c34e1d8b2302, 14, Finished, Available, Finished)

%%sql
select * from ShortcutsMetadata